In [22]:
import os
import time

import pandas as pd
import torch
import tqdm
import numpy as np

from sentence_transformers import SentenceTransformer

In [2]:
query_dev = pd.read_csv("./data/queries/queries.dev.tsv", header=None, sep="\t", index_col=0, names=["query"])
query_dev

,query
1048578,cost of endless pools/swim spa
1048579,what is pcnt
1048580,what is pcb waste
1048581,what is pbis?
1048582,what is paysky
...,...
480594,"price of copper by ounce, pound"
524271,trazodone for dogs side effects
1048565,who plays sebastian michaelis
1048570,what is pearls before swine?


In [3]:
model = SentenceTransformer("all-MiniLM-L6-v2").cuda()

In [6]:
embeddings = model.encode(["a b c d"])

In [33]:
embeddings = []
batch_size, n = 128, len(query_dev)
start = time.time()
for i in tqdm.tqdm(range((n + (batch_size-1)) // batch_size)):
  a, b = batch_size*i, min(n, batch_size*(i+1))
  embedding_i = model.encode(query_dev["query"][a:b].tolist())
  embeddings.append(embedding_i)
print(f"Time Taken: {(time.time()-start):0.4f} s")

100%|██████████| 790/790 [00:16<00:00, 48.83it/s]

Time Taken: 16.1815 s


In [35]:
embeddings_stacked = np.vstack(embeddings)
embeddings_stacked.shape

(101093, 384)

In [38]:
output = "data/embeddings/baseline/embeddings.npy"
os.makedirs(os.path.dirname(output), exist_ok=True)
np.save(output, embeddings_stacked)